In [1]:
# Rishant Pal as part of the team in EE695 course project
!pip install transformers

In [2]:
!pip install -U PyPDF2
!pip install python-docx

In [3]:
import pandas as pd
import numpy as np
import re
from PyPDF2 import PdfReader
import os
import docx

In [4]:
def read_pdf(file_path):
    with open(file_path, "rb") as file:
        pdf_reader = PdfReader(file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_num].extract_text()
    return text

def read_word(file_path):
    doc = docx.Document(file_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

def read_txt(file_path):
    with open(file_path, "r", encoding='utf-8') as file:
        text = file.read()
    return text

def read_documents_from_directory(directory):
    combined_text = ""
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if filename.endswith(".pdf"):
            combined_text += read_pdf(file_path)
        elif filename.endswith(".docx"):
            combined_text += read_word(file_path)
        elif filename.endswith(".txt"):
            combined_text += read_txt(file_path)
    return combined_text

In [5]:
train_directory = '/content'
text_data = read_documents_from_directory(train_directory)
#text_data = re.sub(r'\n+', '\n', text_data).strip()

In [6]:
print(text_data[:100])

 Chapter The family of Dashwood had long been settled in Sussex. Their estate was large, and their 


In [7]:
with open("/train.txt", "w") as f:
    f.write(text_data)

In [8]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [9]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

In [10]:
def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

In [11]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)

  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )

  trainer.train()
  trainer.save_model()

In [12]:
train_file_path = "/train.txt"
model_name = 'gpt2'
output_dir = '/train'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 50
save_steps = 50000

In [13]:
!pip install accelerate -U

In [14]:
!pip install transformers[torch]

In [15]:
pip install transformers[torch]

In [16]:
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
500,3.689700
1000,3.475400
1500,3.341500
2000,3.236900
2500,3.191100
3000,3.061900
3500,3.033500
4000,2.961100
4500,2.882300
5000,2.867500


Step,Training Loss
500,3.689700
1000,3.475400
1500,3.341500
2000,3.236900
2500,3.191100
3000,3.061900
3500,3.033500
4000,2.961100
4500,2.882300
5000,2.867500


In [17]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

In [18]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

def generate_text(model_path, sequence, max_length):

    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [50]:
model1_path = "/train"
sequence1 = "global pandemic impacted the society"
max_len = 100
generate_text(model1_path, sequence1, max_len)

global pandemic impacted the society of poor, sickly girls all over the world. And they are not just any of them. Poor dear girls are at a loss everywhere for any thing to give them better health, or give them any real pleasure, or make them think they are more important than they really are. Girls from about five and twenty are particularly at a great loss in many respects, because they have not got a great many friends. Their home is scarcely within sight of being reckoned quite well


In [25]:
model1_path = "/train"
sequence1 = "Society is all about"
max_len = 150
text=generate_text(model1_path, sequence1, max_len)

Society is all about peace and kindness.” “I know,” said Fanny slowly, “that very well; and though I should never have thought it possible for people of strong feelings to form attachments in such a way as this, it is so, and it will beinfinitely better; forthough there cannot be a war to break them off, ora clergyman to take orders and keep a household, they should always be respectable people, and no profession or housekeeper can set the tone for their feelings if they are not always taking orders from well-meaning strangers. Society, I think, is all that good and generous Do you think it does everybody good? Sure everybody is good and good to some,


In [23]:
model1_path = "/train"
sequence1 = "Detection and Estimation Theory is all regarding "
max_len = 150
generate_text(model1_path, sequence1, max_len)

Detection and Estimation Theory is all regarding everything; it takes three to determine how much a horse or carton of cattle is necessary to a family; so I will give you this hint: ‘If you know of any articles of news to be dropped by Fan - ny, post accordingly;’ and without any being aware of what they are about.’ ‘Are they indeed dropped by Fan- ny?’ ‘No, certainly not.’ ‘Are they delivered to your cousins in Harley Street?’ ‘Almost every body says so; some say more. But we all know that they are not to be settled so near Cleveland.’ ‘Then they are


In [24]:
model1_path = "/train"
sequence1 = "Crimes in society"
max_len = 150
generate_text(model1_path, sequence1, max_len)

Crimes in society are a disgrace’ ‘Oh Lord yes I am very sorry for it. They are very fit to live where they are but little; a scene un - easy in many cases of great similarity; and if I might judge by their manners, the ease, the regularity, the parsonage-age, I mean in their coun - tenance and housekeeping, and not by their high rank and importance.’ Elizabeth could hardly answer her as she proceeded, as she had not an inclination to be longer silent; nor did she feel easy at that moment, when she recollected herself, and saw that the man and woman she had just dis - cussed were exactly such as she had always believed them to


In [27]:
model1_path = "/train"
sequence1 = "Students should do "
max_len = 150
generate_text(model1_path, sequence1, max_len)

Students should do everything in their power to prevent what they think is right, and to clear the names of those they regret, before they are too old and too weak to rise.’ Jane and Elizabeth were all amazement, and some might argue that the care of a family which valued independence above all others would be all that their guardians would allow. ‘But, perhaps,’ said Miss Bingley, ‘independent families may raise too much of a value on a young person of independent fortune to be objectionable.’ ‘This may be true,’ said Darcy, ‘and yet I think independent families are not always so scrupulous as YOURSELF.’ ‘I am not


In [51]:
model1_path = "/train"
sequence1 = "Cherish today, dream of tomorrow"
max_len = 150
generate_text(model1_path, sequence1, max_len)

Cherish today, dream of tomorrow.—But these are not all who you want. Henry will soon be gone. And what is his consequence? That he is laid down by his youngest girl upon the altar, to be devoured and wasted forevermore” “My dear, dear Fanny, how shall I bear it?”—cried Edmund, preventing her by all means from saying,—“There is no bearing any thing at all upon that head. I must feel it.—He has given her away, and we must be cheerful about it together.” Fanny would rather not have been so carried away.—“I should like to know what he thinks,” said Edmund presently, as he folded


In [33]:
model1_path = "/train"
sequence1 = "Revolution signifies"
max_len = 100
generate_text(model1_path, sequence1, max_len)

Revolution signifies something more,’ said Bingley. ‘Something which people look up to, as we have done.’ ‘Perhaps it is as well, perhaps, that Colonel Forster should have had an instrument.’ ‘Perhaps he did not. But as it happens, I thought it was particularly incumbent on him to get into the way, if he could.’ ‘That is a circumstance which Sir John does not get into the way himself. But for


In [35]:
model1_path = "/train"
sequence1 = "Internet can "
max_len = 150
generate_text(model1_path, sequence1, max_len)

Internet can pretend to comprehend more in half a year’s time than mere theoretical applications could comprehend in half a year’ time.’ ‘But indeed these applications, if they are accepted,’ said Mr. Collins, after a short pause, ‘can at least be explained in a few weeks.’ ‘Conjectures, nonsense, nonsense’ cried Elizabeth. ‘Can such be explained in a few weeks’ time?’ ‘They can be explained very easily by means which have not been wholly new to anyone. They can have no basis in common sense; and the same principles, perhaps, may teach us to accept them. But this is a matter which I


In [36]:
model1_path = "/train"
sequence1 = "Poverty can lead to "
max_len = 150
generate_text(model1_path, sequence1, max_len)

Poverty can lead to advantage, and when it does, it is certainly to be a great deal the loss of enjoyment, for so much the worse it always is. When I look back over my own home, and look at all that we have been used to—houses, apartments, roads, streets, everything—it is so abominably vile’ ‘You could argue so,’ replied Elinor; ‘but I would argue nothing else. Edward is only one in a thousand, if Sir John would only allow him five.’ ‘Aye, aye, but THAT is not very vile, indeed, when one comes to compare it with all the rest; and to be


In [41]:
model1_path = "/train"
sequence1 = "Edward says"
max_len = 150
generate_text(model1_path, sequence1, max_len)

Edward says all this, though he may not chuse his own terms, and the intel - ligence given me last night is altogether very agreeable.—I congratulate him.’ ‘Oh he knows his own mind too well, I assure you, and, between ourselves, I have great pleasure in hearing him express himself so warmly on the occasion. He truly delights in being acquainted with the Middletons.’ ‘I know they are not an amiable family, my dear Mr. Dashwood; but as to my poor father’s attachment—I am not sorry to find it is put an end to by hearing what one says of him so affectionately and warmly. Poor man he must really be


In [43]:
model1_path = "/train"
sequence1 = "Artificial intelligence"
max_len = 150
generate_text(model1_path, sequence1, max_len)

Artificial intelligence—no natural talent—to fancy artificial intelligence—and then what can one do?’ ‘I can answer such a question,’ said, smiling. ‘It is not to be supposed that a young man, who had expected to excel, could be without the use of any natural talent. It is evident that he cannot. He must have acquired one, from thorough observation, in order to estimate his equal claims with others.’ ‘I dare say,’ replied Elinor, ‘that his mind is not often so well informed as mine is; or that his memory is not often so assiduous. But I hope,’ she added, ‘that his memory will


In [44]:
model1_path = "/train"
sequence1 = "Mental health"
max_len = 150
generate_text(model1_path, sequence1, max_len)

Mental health is not your thing.’ When she was gone, he repeated his assurance that he would call on Meryton again in the course of the following week, to warm the hearts of both; and Lydia was gratified by such aproceed - ing invitation, and the two parties were again set to work. The plan was that Mrs. Phillips should accompany them to church, and the two girls went together; but as this was not agreed to, Lydia went home in the spirit of her elder sister, to watch for the re - joicing blessings of christiancy, and to throw a sleepless night over every solemn occasion. Chapter Within a short walk of Meryton were the family, in


In [45]:
model1_path = "/train"
sequence1 = "College jobs"
max_len = 150
generate_text(model1_path, sequence1, max_len)

College jobs soon will be open to them.’ But though his father wished them to make many small differences, the difference was not great; and while he talked of ‘our friends at home at War - field,’ and thought it a great pity that they should not all be of equally good neighbours, he thought it a better plan for their employments in general, and con - tinued with his hope of their all being made acquainted with each other through the intimacy. Free eBooks at Planet eBook.comChapter Mr. Bennet’s society was tolerably comfortable from this period of separation; his mother was well provided for; his sisters were all well married, and he had his own


In [49]:
model1_path = "/train"
sequence1 = "College jobs"
max_len = 100
generate_text(model1_path, sequence1, max_len)

College jobs have always at least a certain charm for men.” There was a blush as he said this. “Yes, they have charms for them,” he continued, gathering more courage—“but they must be for real economy.” “Me” cried Tom, striking out a novel idea. “How did you like my idea of real economy?”“I like it as a real economy,” replied the handsome young man,


In [48]:
model1_path = "/train"
sequence1 = "Marraige between "
max_len = 150
generate_text(model1_path, sequence1, max_len)

Marraige between  and HEROCHILL was, asfar as they involved her personally, the most painful to each, and it remained, during the rest of their acquaintance, to be regarded with unqualified approbation. ‘Poor Robert’ said Elinor one day, when they were sitting down to table together at a common table, ‘how unhappy he looks to be going away’— ‘Going about every day,’ replied Marianne; ‘and not coming back in time?’ ‘Oh,’ cried Edward, ‘that is very odd.’ ‘And yet not one of these days seems likely to produce an agree - able marriage between them


In [52]:
model1_path = "/train"
sequence1 = "Upbringing in teens"
max_len = 150
generate_text(model1_path, sequence1, max_len)

Upbringing in teens and young men is not a material business at Highbury. The expense and inconvenience of the young man and woman would be very great on your side, and we should not think of our own expenses any more than we do. You have many, many ideas of what you would like to have done, and I dare say you would have more pride in what you have done than most people. I think about your father a great deal; he is a very liberal landlord, very liberal in his way of doing what he thinks of. When my father died, ten thousand pounds was settled on me as a tenant, and my uncle actually did it himself, which is something very pretty amazing considering the late Mr. Wingfield was not a


In [55]:
model1_path = "/train"
sequence1 = "We should moral because"
max_len = 150
generate_text(model1_path, sequence1, max_len)

We should moral because we feel our own claims, but our own endeavours are to make those claims agreeable. You have, perhaps, heard me speakof love as merely the fashion of a life;—the fashion of wealth. But may you reproach me for presuming that you think only of the fashionable world, and mean to prefer what is more fashion itself to the purest ardours of human life?’ ‘You could not expect me,’ returned Edward, shocked to find himself, under the first glow of Marianne’s tears, to expect even the fashion which Marianne herself had acknowledged to her - self. ‘Your fever was a gown,’ continued Edward, ‘and never attempted conceal


In [58]:
model1_path = "/train"
sequence1 = "Sister's bond"
max_len = 150
generate_text(model1_path, sequence1, max_len)

Sister's bond, and your affection and esteem.’ ‘May I ask to what those feelings were? and was that your engagement to Mr. Wickham?’ ‘Oh yes—the imprudence and folly of my own conduct towards her. To be sanguine on such points But you really shook her head at the foolishness of what I then said, and the folly of my conduct towards her more than once.’ She persisted in this reproach, and for a few moments put an end to it by exclaiming, ‘Oh no, THAT was too much for her. She thought I was teasing her. And when I absolutely re - solved on dawdled away the rest of my


In [60]:
model1_path = "/train"
sequence1 = "My little girl"
max_len = 100
generate_text(model1_path, sequence1, max_len)

My little girl’s coming,’ said. ‘It is very unlucky, for she depends upon your not being troublesome.’ ‘Why, yes, she must, for all practical purposes. If she wanted me to write a note to Mrs. Weston, and request, that it should not be to Mr. Elton, that should not be wanting; I should as soon send it to John and Fan - ny, and they should take care of it. She


In [65]:
model1_path = "/train"
sequence1 = "Sensible men are"
max_len = 100
generate_text(model1_path, sequence1, max_len)

Sensible men are always prudent and quick; always wary. If one is quick and quick, the other quick and dull; if either are not so, the other is worse. But as they advance in knowledge and acquirements, so it shall be equally so. In education there is a material advantage to be gained, and it is better to be by doing well, than to be behindhand, or sitting still and ignorant. Soldiers and sailors need not be always at a loss. They are undoubtedly


In [72]:
model1_path = "/train"
sequence1 = "Every lady in society suffers from"
max_len = 150
generate_text(model1_path, sequence1, max_len)

Every lady in society suffers from this continual recurrence of misfortune.’ He began to speak of her, but Mrs. Weston interrupted him with, ‘You know Miss Fairfax’s excellent taste; she is certainly well worth attending to—and, I believe, one of the most capable, discerning ladies that any can - not exist. She is discerning, undoubtedly.’ ‘I should be extremely sorry to be deficient in any of my acquaintance,’ replied he; ‘but, from my soul, I do not think I shall ever.comknow any thing that I want to know.’ ‘Oh I remember when Mr. Cole once said that ‘dear Mrs. Cole must be


In [76]:
model1_path = "/train"
sequence1 = "Wealth for a poor man is"
max_len = 150
generate_text(model1_path, sequence1, max_len)

Wealth for a poor man is a heavy thing. And as to living upon unfortu - nate land, I do not know that we do not sometimes live where we ought. Poverty, I fear, has much atre - spect to do with it. We ought to be richer, undoubtedly, and I believe it is so. But where, for whatever other claims might exist on either, it would be better for all parties.’ ‘Our circumstances, I trust, will be much more in the line of Providence. Various as they may be—and what is more, uncertain, because of the future—there is still a great deal to be settled before we can be settled—and it will be many years before all


In [78]:
model1_path = "/train"
sequence1 = "playing games are important as"
max_len = 100
generate_text(model1_path, sequence1, max_len)

playing games are important as study and reflection. They teach us to be cautious, to be brooding, to brook roughness and to brook affliction; and they expose us to dreadful impressions of the imprudence and folly of selfish attachment, of needlessuse of time, and the overbearing of inclination. I shall always feel that my own avowal of theprevious is owing to deficiencies in my own education, not to deficiencies of the present—of theprevious period.


In [87]:
model1_path = "/train"
sequence1 = "Matchmakers give advise to"
max_len = 100
generate_text(model1_path, sequence1, max_len)

Matchmakers give advise to Fanny; she is to have the harp fixed and we all know how it will be; I suppose she will get a book instead of her cous - in Richard’s room.’ Lady Bertram did not at present recollect any thing more of the matter. The next morning was damp and windy; but the rain was still excessive, and the party broke out almost entirely. Mr. Rushworth had secured the promise of the ball himself by his


In [99]:
model1_path = "/train"
sequence1 = "Social status means"
max_len = 100
generate_text(model1_path, sequence1, max_len)

Social status means more in - creasing happiness, Mr. Collins found. He felt that Elizabeth must be a disappointment which must sink her even more, and his own disposition told him that his wife was not at all so easily overcome, as to make her at present more wretched. ‘But this is not quite certain;’ said he, in a low voice. ‘In every thing I observed she was worse than last. Very cruelly she treated me with cold politeness yesterday


In [106]:
model1_path = "/train"
sequence1 = "Disparity of wealth affects"
max_len = 200
generate_text(model1_path, sequence1, max_len)

Disparity of wealth affects almost every family. It is most uncommon that Jane Fairfax should have so few friends, but her being single is a blessing.’ ‘I suspect,’ said Mr. Knightley, ‘that single she - self does not think very highly of single women. When she married, she considered it only as a ven - ture, and after receiving several very handsome attentions, it was not clear to her what were the limits of her own regard. Now, however, she seems to feel very decidedly that she must choose to be single.’ ‘Choice—dear Jane Fairfax’ ‘Do you call this a choice?’ said Mr. Knightley warm - ly. ‘Dear me—surely I call it a sentiment which I never had any conception of till very recently. But it is very well suited to a young person. With regard to handsome women, it is very well suited to a young man


In [109]:
model1_path = "/train"
sequence1 = "Marraige is a bond of"
max_len = 150
generate_text(model1_path, sequence1, max_len)

Marraige is a bond of great affection, an - nouncing him every hour of the day.’ ‘And you are doing very well,’ replied Anne, ‘without taking a prodigious pleasure in procuring her good offices. I am half inclined to think Mr Elliot marries a prodigious young man, without being sensible of it.’ ‘Certainly; he is not quite so young, however, without having been a very happy man; and I confess, that if I had known him a fortnight I should not have spoken to him about her. It is a great concession, and one that I am very often very unwilling to do myself; but there is no reason to think that any young wom -


In [111]:
model1_path = "/train"
sequence1 = "Love is"
max_len = 150
generate_text(model1_path, sequence1, max_len)

Love is real, you know—and real friendship—not a little self-possession about you; and love, I really believe, more than common sense—exactly the sort of thing that respectable people should be in love with, in almost every situation excepting public life—and that good kind of steady, regular, never-failing love, which is, you know, so much in demand in society, and so much when there is nothing else to do. Altogether, your conduct is, I think, very par - ticular. You have not been blinded by prejudice or any other bias, or fancying you did not love enough. You know what really matters.’ ‘I know it does. I was


In [113]:
model1_path = "/train"
sequence1 = "Intelligence is really about"
max_len = 150
generate_text(model1_path, sequence1, max_len)

Intelligence is really about finding out which conduct will best suit our ends; and intelligence, after all, has its utility in setting bounds for the rest.’ ‘I cannot agree with you THERE,’ cried Mrs. Weston warmly; ‘I think nothing of the sort HERE. All that I hear is spread abroad, and conjecture everywhere.’ ‘And that you do not think my intel - ligence highly methodical,’ said Mr. Col - lins, looking with lively surprise. ‘My judgment, I fancy, is much more methodical than your own. But I am very liberal on the subject of ALL things. My aunt likes to have her own thoughts to run on; she has


In [116]:
model1_path = "/train"
sequence1 = "Health is important"
max_len = 150
generate_text(model1_path, sequence1, max_len)

Health is important; though sometimes it is really beneficial, and sometimes can prevent its be - ing made indigent, our poor sufferings must be considered as one great relief, and we sympathize in the sight of them even when they are beyond the reach of any creditable support. I should not pretend to imply otherwise, than that we live within compass.’ ‘That is very true,’ replied, ‘and pray tell me how poor you are. For my part, I am only suffering from head colds, and fancying myself as much at home as anybody in the world. If anybody can relieve me, it must be Mrs. Collins.’ She smiled her acceptance; and with a slight bow of the


In [120]:
model1_path = "/train"
sequence1 = "Romantic relationship is good"
max_len = 50
generate_text(model1_path, sequence1, max_len)

Romantic relationship is good. It gives virtue to the person who feels it.’ ‘Yes, there is a virtue in Mr. Darcy’s opinions,’ said Elizabeth with a sneer; ‘but I think
